In [1]:
"""
Блок 2. Оптимизация с помощью физических джойнов
"""

import libs.db_connection

# Hash join
sql1 = """explain (analyze) 
select * 
from block51 a 
inner join block53 b  
on a.user_id = b.user_id 
and a.user_name = b.user_name"""

            
libs.db_connection.select_data_dict_with_commit(sql1, [])

[RealDictRow([('QUERY PLAN',
               'Hash Join  (cost=39189.00..46088.16 rows=1 width=103) (actual time=307.291..307.295 rows=0 loops=1)')]),
 RealDictRow([('QUERY PLAN',
               '  Hash Cond: ((b.user_id = a.user_id) AND ((b.user_name)::text = (a.user_name)::text))')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Seq Scan on block53 b  (cost=0.00..17.80 rows=780 width=76) (actual time=0.585..0.587 rows=10 loops=1)')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Hash  (cost=17353.00..17353.00 rows=1000000 width=27) (actual time=278.103..278.104 rows=1000000 loops=1)')]),
 RealDictRow([('QUERY PLAN',
               '        Buckets: 65536  Batches: 32  Memory Usage: 2347kB')]),
 RealDictRow([('QUERY PLAN',
               '        ->  Seq Scan on block51 a  (cost=0.00..17353.00 rows=1000000 width=27) (actual time=0.524..115.137 rows=1000000 loops=1)')]),
 RealDictRow([('QUERY PLAN', 'Planning Time: 0.420 ms')]),
 RealDictRow([('QUERY PLAN', 'Execution Time: 

In [2]:
# Nested Loop join
sql1 = """explain  (analyze) 
select * 
from block51 a 
inner join block53 b  
on a.user_id = b.user_id 
or a.user_name = b.user_name"""

            
libs.db_connection.select_data_dict_with_commit(sql1, [])

[RealDictRow([('QUERY PLAN',
               'Gather  (cost=1000.00..12305869.77 rows=16736 width=103) (actual time=202.283..1063.986 rows=206 loops=1)')]),
 RealDictRow([('QUERY PLAN', '  Workers Planned: 2')]),
 RealDictRow([('QUERY PLAN', '  Workers Launched: 2')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Nested Loop  (cost=0.00..12303196.17 rows=6973 width=103) (actual time=193.063..1034.287 rows=69 loops=3)')]),
 RealDictRow([('QUERY PLAN',
               '        Join Filter: ((a.user_id = b.user_id) OR ((a.user_name)::text = (b.user_name)::text))')]),
 RealDictRow([('QUERY PLAN', '        Rows Removed by Join Filter: 3333265')]),
 RealDictRow([('QUERY PLAN',
               '        ->  Parallel Seq Scan on block51 a  (cost=0.00..11519.67 rows=416667 width=27) (actual time=0.031..34.422 rows=333333 loops=3)')]),
 RealDictRow([('QUERY PLAN',
               '        ->  Seq Scan on block53 b  (cost=0.00..17.80 rows=780 width=76) (actual time=0.001..0.001 rows=10 loops=100

In [9]:
# Оптимизация 
sql1 = """
explain  (analyze) 
select * 
from block51 a 
inner join block53 b  
on a.user_id = b.user_id

union all

select * 
from block51 a 
inner join block53 b  
on a.user_name = b.user_name
"""

libs.db_connection.select_data_dict_with_commit(sql1, [])

[RealDictRow([('QUERY PLAN',
               'Append  (cost=4.51..14573.68 rows=215 width=103) (actual time=0.035..59.738 rows=206 loops=1)')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Nested Loop  (cost=4.51..475.91 rows=108 width=103) (actual time=0.034..0.264 rows=106 loops=1)')]),
 RealDictRow([('QUERY PLAN',
               '        ->  Seq Scan on block53 b  (cost=0.00..1.10 rows=10 width=76) (actual time=0.003..0.004 rows=10 loops=1)')]),
 RealDictRow([('QUERY PLAN',
               '        ->  Bitmap Heap Scan on block51 a  (cost=4.51..47.37 rows=11 width=27) (actual time=0.007..0.024 rows=11 loops=10)')]),
 RealDictRow([('QUERY PLAN',
               '              Recheck Cond: (user_id = b.user_id)')]),
 RealDictRow([('QUERY PLAN', '              Heap Blocks: exact=106')]),
 RealDictRow([('QUERY PLAN',
               '              ->  Bitmap Index Scan on block11_tst  (cost=0.00..4.51 rows=11 width=0) (actual time=0.003..0.003 rows=11 loops=10)')]),
 RealDictRow([('

In [5]:
# Создаем индекс
sql1 = "create index block11_tst on block51 (user_id, user_name)"

            
libs.db_connection.execute(sql1)

In [6]:
# Создаем индекс
sql1 = "create index block53_tst on block53 (user_id, user_name)"

            
libs.db_connection.execute(sql1)

In [7]:
# Nested Loop join
sql1 = """explain  (analyze) 
select * 
from block51 a 
inner join block53 b  
on a.user_id = b.user_id 
or a.user_name = b.user_name"""

            
libs.db_connection.select_data_dict_with_commit(sql1, [])

[RealDictRow([('QUERY PLAN',
               'Nested Loop  (cost=9046.14..91274.11 rows=215 width=103) (actual time=40.660..236.771 rows=206 loops=1)')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Seq Scan on block53 b  (cost=0.00..1.10 rows=10 width=76) (actual time=0.004..0.017 rows=10 loops=1)')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Bitmap Heap Scan on block51 a  (cost=9046.14..9127.09 rows=21 width=27) (actual time=23.574..23.662 rows=21 loops=10)')]),
 RealDictRow([('QUERY PLAN',
               '        Recheck Cond: ((user_id = b.user_id) OR ((user_name)::text = (b.user_name)::text))')]),
 RealDictRow([('QUERY PLAN', '        Heap Blocks: exact=206')]),
 RealDictRow([('QUERY PLAN',
               '        ->  BitmapOr  (cost=9046.14..9046.14 rows=21 width=0) (actual time=23.542..23.542 rows=0 loops=10)')]),
 RealDictRow([('QUERY PLAN',
               '              ->  Bitmap Index Scan on block11_tst  (cost=0.00..4.51 rows=11 width=0) (actual time=0.012..